In [7]:
import tensorflow as tf
from tensorflow import keras
from scipy.stats import uniform
from keras.models import Sequential
from keras.layers import Embedding, Dropout, LSTM, Dense
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import  RandomizedSearchCV
from collections import Counter
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer

# define the model
def create_lstm_model(units, vocab_length, embedding_matrix, max_len, dropout):
    lstm_model = Sequential()
    embedding_layer = Embedding(vocab_length, 200, weights=[embedding_matrix], input_length=max_len, trainable=False)
    lstm_model.add(embedding_layer)
    lstm_model.add(Dropout(dropout))
    lstm_model.add(LSTM(units))
    lstm_model.add(Dense(5, activation='softmax'))
    lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
    return lstm_model


# fix random seed for reproducibility
#tf.random.set_seed(7)

df = pd.read_csv('../../data/normalized_tweets.csv')
df = df[df['cyberbullying_type'] != 'other_cyberbullying']
# Reset index after filtering out the class
df.reset_index(drop=True, inplace=True)
#df["cyberbullying_type"].value_counts()

### try different length based on tweet lentgh
#df['text_len'] = [len(text.split()) for text in df.tweet_text]
#max_len = np.max(df['text_len'])
max_len = 25

X, y = df["tweet_text"], df["cyberbullying_type"]

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform label encoder on the target variable
y = label_encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#Each word in input used as a key, while a unique index is used as the value of the key 
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(x_train)

X_train = word_tokenizer.texts_to_sequences(x_train)
X_test = word_tokenizer.texts_to_sequences(x_test)

vocab_length = len(word_tokenizer.word_index) + 1

print(vocab_length)

####

X_train = pad_sequences(X_train, padding = 'pre', maxlen = max_len)
X_test = pad_sequences(X_test, padding = 'pre', maxlen = max_len)

# Load GloVe word embeddings and create a dictionary that willl contain words as keys, and their corresponging embedding list as values. 
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('../../glove_embeddings/glove.twitter.27B.200d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_length, 200)) ## change if the dimention of embedding changes above
i = 0
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
    else:
        i = i + 1
print(i)

35868
8150


In [8]:
X_train[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         39, 4017, 1727], dtype=int32)

In [9]:
#import warnings
#warnings.filterwarnings("ignore")


model = KerasClassifier(model=create_lstm_model, units=256, batch_size=64, dropout=0.2, validation_split=0.2,optimizer__learning_rate=0.1, vocab_length=vocab_length, embedding_matrix=embedding_matrix, max_len=max_len)

# Define the grid search parameters
param_grid = dict(optimizer__learning_rate=[0.001,0.01,0.1],
                dropout=[None, 0.1, 0.2, 0.3, 0.5],epochs=[5, 10],
                units=[32,64,128,256],batch_size=[32, 64, 128,256])

# Perform grid search
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,refit=True,cv=5)
grid_result = grid.fit(X_train, y_train)

Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/5
154/154 [==============================] - 3s 16ms/step - loss: 0.6888 - acc: 0.7637 - val_loss: 0.3518 - val_acc: 0.8803
Epoch 2/5
154/154 [==============================] - 2s 15ms/step - loss: 0.3153 - acc: 0.8914 - val_loss: 0.2957 - val_acc: 0.8974
Epoch 3/5
154/154 [==============================] - 2s 15ms/step - loss: 0.2667 - acc: 0.9074 - val_loss: 0.2728 - val_acc: 0.9056
Epoch 4/5
154/154 [==============================] - 2s 15ms/step - loss: 0.2400 - acc: 0.9175 - val_loss: 0.2514 - val_acc: 0.9137
Epoch 5/5
48/48 [==============================] - 0s 5ms/step
Epoch 1/5
154/154 [==============================] - 3s 16ms/step - loss: 0.7269 - acc: 0.7483 - val_loss: 0.3479 - val_acc: 0.8799
Epoch 2/5
154/154 [==============================] - 2s 16ms/step - loss: 0.3232 - acc: 0.8912 - val_loss: 0.2897 - val_acc: 0.8980
Epoch 3/5
154/154 [================

/opt/anaconda3/envs/tf_env4/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tf_env4/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/envs/tf_env4/lib/python3.8/site-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/opt/anaconda3/envs/tf_env4/lib/python3.8/site-packages/scikeras/wrappers.py", line 760, in fit
    self._f

Epoch 1/5
384/384 [==============================] - 24s 59ms/step - loss: 0.3599 - acc: 0.8737 - val_loss: 0.2494 - val_acc: 0.9105
Epoch 2/5
384/384 [==============================] - 24s 62ms/step - loss: 0.2317 - acc: 0.9190 - val_loss: 0.2236 - val_acc: 0.9224
Epoch 3/5
384/384 [==============================] - 22s 59ms/step - loss: 0.1948 - acc: 0.9303 - val_loss: 0.2024 - val_acc: 0.9264
Epoch 4/5
384/384 [==============================] - 24s 61ms/step - loss: 0.1708 - acc: 0.9384 - val_loss: 0.1918 - val_acc: 0.9298
Epoch 5/5
384/384 [==============================] - 22s 58ms/step - loss: 0.1484 - acc: 0.9464 - val_loss: 0.1948 - val_acc: 0.9307


In [10]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.927716 using {'units': 256, 'optimizer__learning_rate': 0.1, 'epochs': 5, 'dropout': 0.2, 'batch_size': 64}
nan (nan) with: {'units': 64, 'optimizer__learning_rate': 0.1, 'epochs': 10, 'dropout': None, 'batch_size': 256}
nan (nan) with: {'units': 256, 'optimizer__learning_rate': 0.1, 'epochs': 10, 'dropout': None, 'batch_size': 128}
0.920096 (0.003263) with: {'units': 32, 'optimizer__learning_rate': 0.001, 'epochs': 5, 'dropout': 0.3, 'batch_size': 128}
nan (nan) with: {'units': 32, 'optimizer__learning_rate': 0.001, 'epochs': 10, 'dropout': None, 'batch_size': 256}
0.926446 (0.004566) with: {'units': 256, 'optimizer__learning_rate': 0.001, 'epochs': 10, 'dropout': 0.3, 'batch_size': 256}
0.914561 (0.002471) with: {'units': 32, 'optimizer__learning_rate': 0.01, 'epochs': 5, 'dropout': 0.3, 'batch_size': 256}
0.927260 (0.003443) with: {'units': 64, 'optimizer__learning_rate': 0.1, 'epochs': 10, 'dropout': 0.2, 'batch_size': 128}
nan (nan) with: {'units': 64, 'optimizer__learning

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.plot(lstm_model_history.history['acc'])
plt.plot(lstm_model_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(lstm_model_history.history['loss'])
plt.plot(lstm_model_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

NameError: name 'lstm_model_history' is not defined